# Dados do Enem por escola

#### Enem por Escola Inicial:

`27 Colunas:`

1. NU_ANO = Ano dos dados escolares coletados.

2. CO_UF_ESCOLA = Unidade Federativa, Estado, onde a escola está localizada.

3. SG_UF_ESCOLA Sigla da Unidade Federativa (ex.: RO = Rondônia).

4. CO_MUNICIPIO_ESCOLA = Código do município onde a escola está localizada.

5. NO_MUNICIPIO_ESCOLA = Nome do município da escola.

6. CO_ESCOLA_EDUCACENSO = Código identificador da escola no Censo Escolar.

7. NO_ESCOLA_EDUCACENSO = Nome oficial da escola.

8. TP_DEPENDENCIA_ADM_ESCOLA = Tipo da dependência administrativa.

9. TP_LOCALIZACAO_ESCOLA = Localização da escola.

10. NU_MATRICULAS = Número total de matrículas na escola naquele ano.

11. NU_PARTICIPANTES_NEC_ESP = Número de participantes com necessidades especiais (quando disponível).

12. NU_PARTICIPANTES = Número total de alunos participantes no ENEM.

13. NU_TAXA_PARTICIPACAO = Porcentagem dos alunos matriculados que participaram do ENEM.

14. NU_MEDIA_CN = Nota média dos alunos em Ciências da Natureza.

15. NU_MEDIA_CH = Nota média dos alunos em Ciências Humanas (ENEM).

16. NU_MEDIA_LP = Nota média dos alunos em Linguagens e Códigos (ENEM).

17. NU_MEDIA_MT = Nota média dos alunos em Matemática (ENEM).

18. NU_MEDIA_RED = Nota média dos alunos na redação (ENEM).

19. NU_MEDIA_OBJ = Média geral das provas objetivas.

20. NU_MEDIA_TOT = Média geral total (incluindo redação e provas objetivas).

21. INSE = Indicador de Nível Socioeconômico da Escola, um índice que mede as condições socioeconômicas.

22. PC_FORMACAO_DOCENTE = Percentual dos professores com formação adequada.

23. NU_TAXA_PERMANENCIA = Taxa percentual dos alunos que permaneceram matriculados na escola até o final do ano letivo.

24. NU_TAXA_APROVACAO = Percentual de alunos aprovados ao fim do ano letivo.

25. NU_TAXA_REPROVACAO = Percentual de alunos reprovados ao fim do ano letivo.

26. NU_TAXA_ABANDONO = Percentual de alunos que abandonaram a escola durante o ano letivo.

27. PORTE_ESCOLA = Classificação da escola pelo número total de alunos matriculados.

In [413]:
# Manipulacao dos dados
import pandas as pd
from unidecode import unidecode
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("ENEM.csv", encoding="latin1", sep=';') 
df.shape

/var/folders/p7/0851733d62n048k3b8qcg4tm0000gn/T/ipykernel_32633/946046593.py:10: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ENEM.csv", encoding="latin1", sep=';')


(172305, 27)

#### Retirar dados desnecessarios:

In [414]:
# Apenas do rio Grande do sul:
df = df[df['SG_UF_ESCOLA'] == 'RS'] # separa só os do RS 

# Apenas a partit de 2012 e ate 2016:
df = df[df['NU_ANO'] > 2011].reset_index(drop=True) 
df = df[df['NU_ANO'] < 2016].reset_index(drop=True) 

#### Retirar colunas denecessarias:

df = df.drop(columns=['CO_UF_ESCOLA']) #Codigo da Unidade Federativa --> podemos tirar pois temos a sigla do estado 
df = df.drop(columns=['CO_ESCOLA_EDUCACENSO']) # Código identificador da escola no Censo Escolar

df.head()

matriculas_por_municipio_ano = df.groupby(['NO_MUNICIPIO_ESCOLA', 'NU_ANO'])['NU_MATRICULAS'].sum().reset_index()

# Mostra os primeiros resultados
print(matriculas_por_municipio_ano.head())

# (Opcional) Salva em um novo CSV
matriculas_por_municipio_ano.to_csv("matriculas_por_municipio_ano.csv", index=False)

  NO_MUNICIPIO_ESCOLA  NU_ANO  NU_MATRICULAS
0              Aceguá    2012             33
1              Aceguá    2014             40
2              Aceguá    2015             63
3               Agudo    2012             97
4               Agudo    2013            106


### 2. Uniformização dos Dados:
- Padronizar valores Categoricos;
- Padronizar valores numéricos.

In [415]:
# Dados categóricos para limpeza:
colunas_to_clean = [
    'SG_UF_ESCOLA', 'NO_MUNICIPIO_ESCOLA',
    'NO_ESCOLA_EDUCACENSO',
    'PORTE_ESCOLA'
]

for col in colunas_to_clean:
    df[col] = (
        df[col].astype(str)
               .str.upper()
               .str.strip()
               .str.replace('-', '', regex=False)
               .str.replace(' ', '_', regex=False)
               .str.replace('�', '/', regex=False)
               .str.replace(r'\s+', ' ', regex=True)
    )

# Dados Numericos:
colunas_to_clean = [
    'NU_ANO',
    'CO_MUNICIPIO_ESCOLA',
    'TP_LOCALIZACAO_ESCOLA',
    'NU_MATRICULAS',
    'NU_PARTICIPANTES_NEC_ESP',
    'NU_PARTICIPANTES',
    'NU_TAXA_PARTICIPACAO',
    'TP_DEPENDENCIA_ADM_ESCOLA',
    'NU_MEDIA_CN',
    'NU_MEDIA_CH',
    'NU_MEDIA_LP',
    'NU_MEDIA_MT',
    'NU_MEDIA_RED',
    'NU_MEDIA_OBJ',
    'NU_MEDIA_TOT',
    'INSE',
    'PC_FORMACAO_DOCENTE',
    'NU_TAXA_PERMANENCIA',
    'NU_TAXA_APROVACAO',
    'NU_TAXA_REPROVACAO',
    'NU_TAXA_ABANDONO'
]

for col in colunas_to_clean:
    df[col] = pd.to_numeric(
        df[col].astype(str)
                .str.strip()
                .str.replace(' ', '', regex=False)
                .str.replace(',', '', regex=False),
        errors='coerce'  # substitui valores inválidos por NaN automaticamente
    )

### 3. Análise Exploratoria dos Dados:
- Indentificar a qualidade dos dados; 
- Identificar dados ausentes;
- Tratamento desses Dados.

In [416]:
#Mostra a % de valores faltantes: round(3.141592653589793, 2)
def Faltantes():
    faltantes = df.isna().sum()
    porcentagem = round(((faltantes / len(df)) * 100), 2)
    resultado = pd.DataFrame({
        'Qtd. Faltantes': faltantes,
        '% Faltantes': porcentagem
    }).sort_values(by='% Faltantes', ascending=False)

    print(resultado)

#### Procura por dados duplicados:

In [417]:
#Nao temos nenhum dado duplicado nesse data frame
df[df.duplicated()]


,NU_ANO,SG_UF_ESCOLA,CO_MUNICIPIO_ESCOLA,NO_MUNICIPIO_ESCOLA,NO_ESCOLA_EDUCACENSO,TP_DEPENDENCIA_ADM_ESCOLA,TP_LOCALIZACAO_ESCOLA,NU_MATRICULAS,NU_PARTICIPANTES_NEC_ESP,NU_PARTICIPANTES,...,NU_MEDIA_RED,NU_MEDIA_OBJ,NU_MEDIA_TOT,INSE,PC_FORMACAO_DOCENTE,NU_TAXA_PERMANENCIA,NU_TAXA_APROVACAO,NU_TAXA_REPROVACAO,NU_TAXA_ABANDONO,PORTE_ESCOLA


In [418]:
df.describe()

,NU_ANO,CO_MUNICIPIO_ESCOLA,TP_DEPENDENCIA_ADM_ESCOLA,TP_LOCALIZACAO_ESCOLA,NU_MATRICULAS,NU_PARTICIPANTES_NEC_ESP,NU_PARTICIPANTES,NU_TAXA_PARTICIPACAO,NU_MEDIA_CN,NU_MEDIA_CH,...,NU_MEDIA_MT,NU_MEDIA_RED,NU_MEDIA_OBJ,NU_MEDIA_TOT,INSE,PC_FORMACAO_DOCENTE,NU_TAXA_PERMANENCIA,NU_TAXA_APROVACAO,NU_TAXA_REPROVACAO,NU_TAXA_ABANDONO
count,4195.000000,4.195000e+03,4195.000000,4195.000000,4195.000000,3270.000000,4195.000000,4195.000000,4195.000000,4195.000000,...,4195.000000,4195.000000,0.0,0.0,0.0,3270.000000,2233.000000,4194.000000,4194.000000,4194.000000
mean,2013.578784,4.312370e+06,2.508701,1.047914,71.375209,0.353823,50.674136,72.937416,490.964031,547.632007,...,516.556119,534.414031,NaN,NaN,NaN,63.793914,85.176982,81.608774,12.726276,5.664950
std,1.102379,6.338968e+03,0.903080,0.213610,59.557175,0.805994,41.804783,13.518618,38.926055,40.560200,...,64.205506,66.575331,NaN,NaN,NaN,13.563914,14.029989,12.368188,9.290319,6.252032
min,2012.000000,4.300034e+06,1.000000,1.000000,10.000000,0.000000,10.000000,50.000000,407.850000,420.780000,...,381.570000,262.670000,NaN,NaN,NaN,0.000000,0.000000,31.800000,0.000000,0.000000
25%,2013.000000,4.306700e+06,2.000000,1.000000,30.000000,0.000000,21.000000,61.935000,464.570000,517.395000,...,468.375000,490.825000,NaN,NaN,NaN,54.500000,80.880000,73.000000,5.600000,0.000000
50%,2014.000000,4.314076e+06,2.000000,1.000000,51.000000,0.000000,37.000000,72.000000,480.220000,545.350000,...,505.090000,525.540000,NaN,NaN,NaN,63.400000,88.000000,83.600000,10.950000,3.800000
75%,2015.000000,4.316907e+06,4.000000,1.000000,94.000000,0.000000,66.000000,83.330000,509.380000,571.260000,...,551.340000,571.120000,NaN,NaN,NaN,73.700000,93.750000,91.900000,18.100000,9.100000
max,2015.000000,4.323804e+06,4.000000,2.000000,551.000000,10.000000,360.000000,100.000000,686.450000,686.490000,...,767.740000,820.000000,NaN,NaN,NaN,100.000000,100.000000,100.000000,62.700000,38.700000


In [419]:
Faltantes()
df.shape

                           Qtd. Faltantes  % Faltantes
INSE                                 4195       100.00
NU_MEDIA_TOT                         4195       100.00
NU_MEDIA_OBJ                         4195       100.00
NU_TAXA_PERMANENCIA                  1962        46.77
PC_FORMACAO_DOCENTE                   925        22.05
NU_PARTICIPANTES_NEC_ESP              925        22.05
NU_TAXA_ABANDONO                        1         0.02
NU_TAXA_REPROVACAO                      1         0.02
NU_TAXA_APROVACAO                       1         0.02
NU_ANO                                  0         0.00
NU_MEDIA_LP                             0         0.00
NU_MEDIA_RED                            0         0.00
NU_MEDIA_MT                             0         0.00
NU_MEDIA_CH                             0         0.00
SG_UF_ESCOLA                            0         0.00
NU_MEDIA_CN                             0         0.00
NU_TAXA_PARTICIPACAO                    0         0.00
NU_PARTICI

(4195, 25)

#### Retirar colunas denecessarias ou Subistituir por outro valor valido:

In [420]:
#NU_MEDIA_OBJ: 

# Identifica onde NU_MEDIA_OBJ está faltando e as outras 4 médias estão presentes
condicao = df['NU_MEDIA_OBJ'].isna() & df[['NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT']].notna().all(axis=1)

# Calcula a média das 4 áreas objetivas e preenche os valores faltantes
df.loc[condicao, 'NU_MEDIA_OBJ'] = df.loc[condicao, ['NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT']].mean(axis=1)

In [421]:
#NU_MEDIA_TOT

# Condição para identificar onde NU_MEDIA_TOT é NaN, mas NU_MEDIA_OBJ e NU_MEDIA_RED são válidos
condicao_tot = df['NU_MEDIA_TOT'].isna() & df[['NU_MEDIA_OBJ', 'NU_MEDIA_RED']].notna().all(axis=1)

# Preenchendo NU_MEDIA_TOT usando a média ponderada das áreas objetivas e redação
df.loc[condicao_tot, 'NU_MEDIA_TOT'] = (
    (df.loc[condicao_tot, 'NU_MEDIA_OBJ'] * 4 + df.loc[condicao_tot, 'NU_MEDIA_RED']) / 5)

Faltantes()

                           Qtd. Faltantes  % Faltantes
INSE                                 4195       100.00
NU_TAXA_PERMANENCIA                  1962        46.77
PC_FORMACAO_DOCENTE                   925        22.05
NU_PARTICIPANTES_NEC_ESP              925        22.05
NU_TAXA_ABANDONO                        1         0.02
NU_TAXA_REPROVACAO                      1         0.02
NU_TAXA_APROVACAO                       1         0.02
NU_ANO                                  0         0.00
NU_MEDIA_LP                             0         0.00
NU_MEDIA_TOT                            0         0.00
NU_MEDIA_OBJ                            0         0.00
NU_MEDIA_RED                            0         0.00
NU_MEDIA_MT                             0         0.00
NU_MEDIA_CH                             0         0.00
SG_UF_ESCOLA                            0         0.00
NU_MEDIA_CN                             0         0.00
NU_TAXA_PARTICIPACAO                    0         0.00
NU_PARTICI

`Retirar os dados ausantes:`

INSE - 100% faltando 

NU_TAXA_ABANDONO - apenas 1 faltando 

NU_TAXA_REPROVACAO - apenas 1 faltando

NU_TAXA_APROVACAO - apenas 1 faltando

In [422]:
df = df.drop(columns=['INSE']) 

index = df[df['NU_TAXA_ABANDONO'].isna()].index
df.drop(index=index, inplace=True)

index = df[df['NU_TAXA_REPROVACAO'].isna()].index
df.drop(index=index, inplace=True)

index = df[df['NU_TAXA_APROVACAO'].isna()].index
df.drop(index=index, inplace=True)

Faltantes()

                           Qtd. Faltantes  % Faltantes
NU_TAXA_PERMANENCIA                  1961        46.76
PC_FORMACAO_DOCENTE                   924        22.03
NU_PARTICIPANTES_NEC_ESP              924        22.03
NU_ANO                                  0         0.00
NU_MEDIA_LP                             0         0.00
NU_TAXA_ABANDONO                        0         0.00
NU_TAXA_REPROVACAO                      0         0.00
NU_TAXA_APROVACAO                       0         0.00
NU_MEDIA_TOT                            0         0.00
NU_MEDIA_OBJ                            0         0.00
NU_MEDIA_RED                            0         0.00
NU_MEDIA_MT                             0         0.00
NU_MEDIA_CH                             0         0.00
SG_UF_ESCOLA                            0         0.00
NU_MEDIA_CN                             0         0.00
NU_TAXA_PARTICIPACAO                    0         0.00
NU_PARTICIPANTES                        0         0.00
NU_MATRICU

`Tentar encontrar uma correlacao:`

NU_TAXA_PERMANENCIA - 46% faltando

PC_FORMACAO_DOCENTE - 22% faltando 

NU_PARTICIPANTES_NEC_ESP - 22% faltando



In [423]:
# Calcula correlação apenas para colunas numéricas
corr_numericas = df.select_dtypes(include=['number']).corr()

# Exibe correlação apenas da coluna NU_TAXA_PERMANENCIA com as outras
corr_numericas['NU_TAXA_PERMANENCIA'].sort_values(ascending=False)

NU_TAXA_PERMANENCIA          1.000000
NU_TAXA_APROVACAO            0.134088
TP_LOCALIZACAO_ESCOLA        0.071395
NU_TAXA_PARTICIPACAO        -0.012912
NU_PARTICIPANTES_NEC_ESP    -0.019913
CO_MUNICIPIO_ESCOLA         -0.022444
NU_MEDIA_MT                 -0.024762
NU_MEDIA_CN                 -0.026876
PC_FORMACAO_DOCENTE         -0.031188
NU_ANO                      -0.035309
NU_MEDIA_OBJ                -0.050211
NU_MEDIA_TOT                -0.053335
NU_MEDIA_RED                -0.053802
NU_MATRICULAS               -0.057789
NU_PARTICIPANTES            -0.059146
NU_MEDIA_CH                 -0.062859
NU_TAXA_ABANDONO            -0.074995
NU_MEDIA_LP                 -0.100932
NU_TAXA_REPROVACAO          -0.125791
TP_DEPENDENCIA_ADM_ESCOLA   -0.175382
Name: NU_TAXA_PERMANENCIA, dtype: float64

In [424]:
corr_numericas = df.select_dtypes(include=['number']).corr()
corr_numericas['PC_FORMACAO_DOCENTE'].sort_values(ascending=False)

PC_FORMACAO_DOCENTE          1.000000
NU_MEDIA_OBJ                 0.339601
NU_MEDIA_TOT                 0.339034
NU_MEDIA_LP                  0.330423
NU_MEDIA_CN                  0.321337
NU_MEDIA_MT                  0.300880
NU_MEDIA_CH                  0.298859
NU_MEDIA_RED                 0.294916
NU_TAXA_PARTICIPACAO         0.259853
TP_DEPENDENCIA_ADM_ESCOLA    0.173915
NU_PARTICIPANTES             0.164389
NU_MATRICULAS                0.104664
CO_MUNICIPIO_ESCOLA          0.096003
NU_TAXA_APROVACAO            0.064787
NU_TAXA_REPROVACAO           0.020042
NU_PARTICIPANTES_NEC_ESP     0.002464
NU_ANO                      -0.019935
NU_TAXA_PERMANENCIA         -0.031188
TP_LOCALIZACAO_ESCOLA       -0.059882
NU_TAXA_ABANDONO            -0.165110
Name: PC_FORMACAO_DOCENTE, dtype: float64

In [425]:
corr_numericas = df.select_dtypes(include=['number']).corr()
corr_numericas['NU_PARTICIPANTES_NEC_ESP'].sort_values(ascending=False)

NU_PARTICIPANTES_NEC_ESP     1.000000
NU_MATRICULAS                0.273129
NU_PARTICIPANTES             0.257054
NU_ANO                       0.200180
NU_TAXA_REPROVACAO           0.109325
NU_TAXA_ABANDONO             0.102389
PC_FORMACAO_DOCENTE          0.002464
NU_MEDIA_CH                 -0.000439
CO_MUNICIPIO_ESCOLA         -0.016043
NU_TAXA_PERMANENCIA         -0.019913
NU_MEDIA_RED                -0.045294
NU_MEDIA_LP                 -0.058533
TP_LOCALIZACAO_ESCOLA       -0.068946
NU_TAXA_PARTICIPACAO        -0.069818
NU_MEDIA_TOT                -0.080912
NU_MEDIA_OBJ                -0.091744
NU_MEDIA_CN                 -0.105734
TP_DEPENDENCIA_ADM_ESCOLA   -0.115106
NU_TAXA_APROVACAO           -0.134135
NU_MEDIA_MT                 -0.138252
Name: NU_PARTICIPANTES_NEC_ESP, dtype: float64

`Nenhum desses temos uma coorelacao forte o suficiente. `

In [426]:
df.describe()

,NU_ANO,CO_MUNICIPIO_ESCOLA,TP_DEPENDENCIA_ADM_ESCOLA,TP_LOCALIZACAO_ESCOLA,NU_MATRICULAS,NU_PARTICIPANTES_NEC_ESP,NU_PARTICIPANTES,NU_TAXA_PARTICIPACAO,NU_MEDIA_CN,NU_MEDIA_CH,NU_MEDIA_LP,NU_MEDIA_MT,NU_MEDIA_RED,NU_MEDIA_OBJ,NU_MEDIA_TOT,PC_FORMACAO_DOCENTE,NU_TAXA_PERMANENCIA,NU_TAXA_APROVACAO,NU_TAXA_REPROVACAO,NU_TAXA_ABANDONO
count,4194.000000,4.194000e+03,4194.000000,4194.000000,4194.000000,3270.000000,4194.000000,4194.000000,4194.00000,4194.000000,4194.000000,4194.000000,4194.000000,4194.000000,4194.000000,3270.000000,2233.000000,4194.000000,4194.000000,4194.000000
mean,2013.579161,4.312370e+06,2.509061,1.047926,71.360277,0.353823,50.659990,72.935234,490.96031,547.632663,513.755405,516.547816,534.412830,517.224049,520.661805,63.793914,85.176982,81.608774,12.726276,5.664950
std,1.102240,6.339645e+03,0.902887,0.213634,59.556423,0.805994,41.799726,13.519491,38.92995,40.565014,35.864976,64.210909,66.583224,41.026983,44.445187,13.563914,14.029989,12.368188,9.290319,6.252032
min,2012.000000,4.300034e+06,1.000000,1.000000,10.000000,0.000000,10.000000,50.000000,407.85000,420.780000,402.540000,381.570000,262.670000,413.477500,400.636000,0.000000,0.000000,31.800000,0.000000,0.000000
25%,2013.000000,4.306700e+06,2.000000,1.000000,30.000000,0.000000,21.000000,61.917500,464.57000,517.392500,489.460000,468.372500,490.822500,489.208750,490.073000,54.500000,80.880000,73.000000,5.600000,0.000000
50%,2014.000000,4.314072e+06,2.000000,1.000000,51.000000,0.000000,37.000000,72.000000,480.20500,545.350000,507.295000,505.075000,525.530000,505.113750,508.999000,63.400000,88.000000,83.600000,10.950000,3.800000
75%,2015.000000,4.316907e+06,4.000000,1.000000,94.000000,0.000000,66.000000,83.330000,509.39000,571.270000,535.680000,551.317500,571.160000,538.266875,543.928500,73.700000,93.750000,91.900000,18.100000,9.100000
max,2015.000000,4.323804e+06,4.000000,2.000000,551.000000,10.000000,360.000000,100.000000,686.45000,686.490000,643.470000,767.740000,820.000000,690.265000,716.212000,100.000000,100.000000,100.000000,62.700000,38.700000


In [427]:
df.shape

(4194, 24)

### Qualidade dos dados:

`Pontos positivos:` 
- Mais de 4 mil registros.

`Pontos negativos:` 
- Altíssima dispersão em algumas colunas como a do numero de participantes. 

#### Outliers:
Por causa da alta dispercao dos dados, nos nao conseguimos indentificar nenhum dado que posso ser considerado um outlier. 

Para visulaizar o grafico só rode o segunte codigo:{

sns.kdeplot(df['NU_MATRICULAS']) 
plt.xlabel('Quantidade de matriculados')
plt.ylabel('Dencidade')
plt.title('Numero de matriculados')
plt.show()


}

#### Remover as modalidades de ensino que não fazem parte do escopo do nosso trabalho

1 - Federal, 
2 - Estadual,
3 - Municipal,
4 - Privada.

In [ ]:
#Filtrar o DataFrame onde o valor da coluna 'TP_DEPENDENCIA_ADM_ESCOLA' para ser 1, 2 ou 3:
df = df[df['TP_DEPENDENCIA_ADM_ESCOLA'].isin([1, 2, 3])]

print(df['TP_DEPENDENCIA_ADM_ESCOLA'])

#df = df.drop(columns=['CO_UF_ESCOLA']) #Codigo da Unidade Federativa --> podemos tirar pois temos a sigla do estado 

#df = df.drop(columns=['CO_ESCOLA_EDUCACENSO']) # Código identificador da escola no Censo Escolar

df.head()

matriculas_por_municipio_ano = df.groupby(['NO_MUNICIPIO_ESCOLA', 'NU_ANO'])['NU_MATRICULAS'].sum().reset_index()

# Mostra os primeiros resultados
print(matriculas_por_municipio_ano.head())

# (Opcional) Salva em um novo CSV
matriculas_por_municipio_ano.to_csv("matriculas_por_municipio_ano_sem_priv.csv", index=False)

0       1
1       1
2       1
3       1
4       2
       ..
4190    1
4191    1
4192    1
4193    1
4194    1
Name: TP_DEPENDENCIA_ADM_ESCOLA, Length: 3095, dtype: int64
  NO_MUNICIPIO_ESCOLA  NU_ANO  NU_MATRICULAS
0              ACEGUÁ    2012             33
1              ACEGUÁ    2014             40
2              ACEGUÁ    2015             63
3               AGUDO    2012             97
4               AGUDO    2013            106


#### Para fazer a juncao das duas tabelas temos q agrupar por municipio e por ano

In [429]:

df = df.groupby(['NO_MUNICIPIO_ESCOLA', 'NU_ANO']).agg({
    'NU_MATRICULAS': 'sum',
    'NU_PARTICIPANTES': 'sum',
    'NU_PARTICIPANTES_NEC_ESP': 'sum',
    'NU_TAXA_PARTICIPACAO': 'mean',
    'NU_MEDIA_CN': 'mean',
    'NU_MEDIA_CH': 'mean',
    'NU_MEDIA_LP': 'mean',
    'NU_MEDIA_MT': 'mean',
    'NU_MEDIA_RED': 'mean',
    'NU_MEDIA_OBJ': 'mean',
    'NU_MEDIA_TOT': 'mean',
    'PC_FORMACAO_DOCENTE': 'mean',
    'NU_TAXA_PERMANENCIA': 'mean',
    'NU_TAXA_APROVACAO': 'mean',
    'NU_TAXA_REPROVACAO': 'mean',
    'NU_TAXA_ABANDONO': 'mean'
}).reset_index()

df = df.round(2)

df = df.rename(columns={
    'NO_MUNICIPIO_ESCOLA': 'Municipio',
    'NU_ANO': 'Ano'
})

#### Retiriar Colunas desnecessarias 

In [430]:
df = df.drop(columns=['NU_MEDIA_OBJ'])
df = df.drop(columns=['NU_MEDIA_RED'])
df = df.drop(columns=['NU_MEDIA_MT'])
df = df.drop(columns=['NU_MEDIA_LP'])
df = df.drop(columns=['NU_MEDIA_CN'])
df = df.drop(columns=['NU_MEDIA_CH'])
df = df.drop(columns=['NU_PARTICIPANTES_NEC_ESP'])
df = df.drop(columns=['PC_FORMACAO_DOCENTE'])
df = df.drop(columns=['NU_TAXA_APROVACAO'])
df = df.drop(columns=['NU_TAXA_PARTICIPACAO'])
df = df.drop(columns=['NU_TAXA_REPROVACAO'])


print(df.columns)

Index(['Municipio', 'Ano', 'NU_MATRICULAS', 'NU_PARTICIPANTES', 'NU_MEDIA_TOT',
       'NU_TAXA_PERMANENCIA', 'NU_TAXA_ABANDONO'],
      dtype='object')


#### Salva em um csv 

In [431]:
# Salvar os dados: 
df.to_csv("df1.csv", index=False)